In [1]:
# clone repo
!rm -rf gpt_from_scratch
!git clone https://github.com/b-schoen/gpt_from_scratch.git

# change into the repo directory
import os

os.chdir('gpt_from_scratch')

print("Current Working Directory:", os.getcwd())


Cloning into 'gpt_from_scratch'...
remote: Enumerating objects: 489, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 489 (delta 60), reused 92 (delta 26), pack-reused 352 (from 1)
Receiving objects: 100% (489/489), 15.11 MiB | 30.52 MiB/s, done.
Resolving deltas: 100% (252/252), done.
Current Working Directory: /content/gpt_from_scratch


In [2]:
try:
  # for google colab users
    import google.colab # type: ignore
    from google.colab import output
    COLAB = True
    %pip install sae-lens transformer-lens jaxtyping
except:
  # for local setup
    COLAB = False
    from IPython import get_ipython # type: ignore
    ipython = get_ipython(); assert ipython is not None
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.0/920.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.8/174.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 65.4 MB/s eta 0:00:00
   ━━

In [3]:
import torch
import transformer_lens

# TODO(bschoen): Just start using `transformer_lens.utils.get_device` from now on
def get_best_available_torch_device() -> torch.device:
    return transformer_lens.utils.get_device()

In [4]:
from jaxtyping import Float32

In [5]:
from gpt_from_scratch import python_utils

In [6]:
# imports from https://github.com/jbloomAus/SAELens/blob/main/tutorials/tutorial_2_0.ipynb
import os
import dataclasses

import torch
from tqdm import tqdm
import plotly.express as px
import pandas as pd
import tabulate

In [7]:
# disable autograd, as we're focused on inference and this save us a lot of speed, memory, and annoying boilerplate
torch.set_grad_enabled(False)

In [8]:
# note: jbloom advice is to make a run_comparer here: https://docs.wandb.ai/guides/app/features/panels/run-comparer

## Loading A Pretrained Sparse Autoencoder

In practice, SAEs can be of varying usefulness for general use cases. To start with, we recommend the following:

* Joseph's Open Source GPT2 Small Residual (gpt2-small-res-jb)
* Joseph's Feature Splitting (gpt2-small-res-jb-feature-splitting)
* Gemma SAEs (gemma-2b-res-jb) (0,6) <- on Neuronpedia and good. (12 / 17 aren't very good currently).

Other SAEs have various issues--e.g., too dense or not dense enough, or designed for special use cases, or initial drafts of what we hope will be better versions later. Decode Research / Neuronpedia are working on making all SAEs on Neuronpedia loadable in SAE Lens and vice versa, as well as providing public benchmarking stats to help people choose which SAEs to work with.

To see all the SAEs contained in a specific release (named after the part of the model they apply to), simply run the below. Each hook point corresponds to a layer or module of the model.

In [9]:
import sae_lens.toolkit
import sae_lens.toolkit.pretrained_saes_directory
import sae_lens.toolkit.pretrained_sae_loaders

from sae_lens.toolkit.pretrained_saes_directory import PretrainedSAELookup

In [10]:
print("Pretrained SAE loaders:")
for name in sae_lens.toolkit.pretrained_sae_loaders.NAMED_PRETRAINED_SAE_LOADERS.keys():
    print(f" - {name}")

Pretrained SAE loaders:
 - sae_lens
 - connor_rob_hook_z
 - gemma_2


In [11]:
# loads from `pretrained_saes.yaml`
pretrained_saes_dir: dict[str, PretrainedSAELookup] = (
    sae_lens.toolkit.pretrained_saes_directory.get_pretrained_saes_directory()
)

print(f"Found {len(pretrained_saes_dir)} pretrained SAEs")
df = pd.DataFrame([dataclasses.asdict(x) for x in pretrained_saes_dir.values()])


df = df[["repo_id", "release", "model"]]

df.sort_values(by=df.columns.to_list())

Found 40 pretrained SAEs


,repo_id,release,model
8,JoshEngels/Mistral-7B-Residual-Stream-SAEs,mistral-7b-res-wg,mistral-7b
1,ckkissane/attn-saes-gpt2-small-all-layers,gpt2-small-hook-z-kk,gpt2-small
33,ctigges/pythia-70m-deduped__att-sm_processed,pythia-70m-deduped-att-sm,pythia-70m-deduped
32,ctigges/pythia-70m-deduped__mlp-sm_processed,pythia-70m-deduped-mlp-sm,pythia-70m-deduped
31,ctigges/pythia-70m-deduped__res-sm_processed,pythia-70m-deduped-res-sm,pythia-70m-deduped
29,google/gemma-scope-27b-pt-res,gemma-scope-27b-pt-res,gemma-2-27b
30,google/gemma-scope-27b-pt-res,gemma-scope-27b-pt-res-canonical,gemma-2-27b
19,google/gemma-scope-2b-pt-att,gemma-scope-2b-pt-att,gemma-2-2b
20,google/gemma-scope-2b-pt-att,gemma-scope-2b-pt-att-canonical,gemma-2-2b
17,google/gemma-scope-2b-pt-mlp,gemma-scope-2b-pt-mlp,gemma-2-2b


In [12]:
# let's look at which ones are there for gemma-2b
df[df["model"] == "gemma-2-2b"][["release", "repo_id"]].sort_values(
    by=["release", "repo_id"]
)

,release,repo_id
19,gemma-scope-2b-pt-att,google/gemma-scope-2b-pt-att
20,gemma-scope-2b-pt-att-canonical,google/gemma-scope-2b-pt-att
17,gemma-scope-2b-pt-mlp,google/gemma-scope-2b-pt-mlp
18,gemma-scope-2b-pt-mlp-canonical,google/gemma-scope-2b-pt-mlp
15,gemma-scope-2b-pt-res,google/gemma-scope-2b-pt-res
16,gemma-scope-2b-pt-res-canonical,google/gemma-scope-2b-pt-res


In [13]:
# We'll use this one, since it's what's used in the GemmaScope tutorial
pretrained_sae_name = (
    "gemma-scope-2b-pt-res"  # repo_id = `google/gemma-scope-2b-pt-res`
)

# pretrained_sae_name = "gemma-scope-2b-pt-res-canonical"

In [14]:
# note: `"saes_map"` maps `<sae-id>: <hook-point>`
pretrained_sae_lookup: PretrainedSAELookup = pretrained_saes_dir[pretrained_sae_name]

# note: only layers 5, 12, and 19 seem to have the 1m width
python_utils.print_json(pretrained_sae_lookup)

{
  "release": "gemma-scope-2b-pt-res",
  "repo_id": "google/gemma-scope-2b-pt-res",
  "model": "gemma-2-2b",
  "conversion_func": "gemma_2",
  "saes_map": {
    "layer_0/width_16k/average_l0_105": "layer_0/width_16k/average_l0_105",
    "layer_0/width_16k/average_l0_13": "layer_0/width_16k/average_l0_13",
    "layer_0/width_16k/average_l0_226": "layer_0/width_16k/average_l0_226",
    "layer_0/width_16k/average_l0_25": "layer_0/width_16k/average_l0_25",
    "layer_0/width_16k/average_l0_46": "layer_0/width_16k/average_l0_46",
    "layer_1/width_16k/average_l0_10": "layer_1/width_16k/average_l0_10",
    "layer_1/width_16k/average_l0_102": "layer_1/width_16k/average_l0_102",
    "layer_1/width_16k/average_l0_20": "layer_1/width_16k/average_l0_20",
    "layer_1/width_16k/average_l0_250": "layer_1/width_16k/average_l0_250",
    "layer_1/width_16k/average_l0_40": "layer_1/width_16k/average_l0_40",
    "layer_2/width_16k/average_l0_13": "layer_2/width_16k/average_l0_13",
    "layer_2/width_1

In [15]:
# we choose:
# - last layer
# - largest available width
# - lowest l0 sparsity "on average, how many neurons (features for SAEs) activate"
# sae_id = 'layer_25/width_65k/average_l0_15'

# actually need to choose biggest one with autointerp explanations evailable
#
# we'll use the exact naming from the `Getting Started With Gemma` notebook: https://colab.research.google.com/drive/17dQFYUYnuKnP6OwQPH9v_GSYUW5aj-Rp?usp=sharing#scrollTo=BP2Ju5AnNIzS
# this is required for neuronpedia to match
#
# sae_id = 'layer_25/width_16k/average_l0_16'

# canonical seem to line up better with the neuronpedia names
# sae_id = "layer_25/width_16k/canonical"

# we'll try exact one from tutorial
sae_id = "layer_20/width_16k/average_l0_71"

# - it's not the one from the metadata
# - note: literally none of these work
#
# sae_id = "layer_25/width_16k/average_l0_116"
# sae_id = "layer_25/width_16k/average_l0_16" # TODO(bschoen): the one from the metadata really should work
# sae_id = "layer_25/width_16k/average_l0_28"
# sae_id = "layer_25/width_16k/average_l0_55"
# sae_id = "layer_25/width_16k/average_l0_285"

In [16]:
device = get_best_available_torch_device()

# the cfg dict is returned alongside the SAE since it may contain useful information for analysing the SAE (eg: instantiating an activation store)
# Note that this is not the same as the SAEs config dict, rather it is whatever was in the HF repo, from which we can extract the SAE config dict
# We also return the feature sparsities which are stored in HF for convenience.
sae, cfg_dict, sparsity = sae_lens.SAE.from_pretrained(
    release=pretrained_sae_name,  # <- Release name
    sae_id=sae_id,  # <- SAE id (not always a hook point!)
    device=device,
)

params.npz:   0%|          | 0.00/302M [00:00<?, ?B/s]

In [17]:
# split out into types for readability
sae: sae_lens.SAE = sae
cfg_dict: dict[str, str | int | None | torch.device | bool] = cfg_dict
sparsity: dict = sparsity

In [18]:
print("Show config (since usually small)")
cfg_dict

Show config (since usually small)


{'architecture': 'jumprelu',
 'd_in': 2304,
 'd_sae': 16384,
 'dtype': 'float32',
 'model_name': 'gemma-2-2b',
 'hook_name': 'blocks.20.hook_resid_post',
 'hook_layer': 20,
 'hook_head_index': None,
 'activation_fn_str': 'relu',
 'finetuning_scaling_factor': False,
 'sae_lens_training_version': None,
 'prepend_bos': True,
 'dataset_path': 'monology/pile-uncopyrighted',
 'context_size': 1024,
 'dataset_trust_remote_code': True,
 'apply_b_dec_to_input': False,
 'normalize_activations': None,
 'device': device(type='cuda')}

In [19]:
# note: sparsity is average l0 sparsity? is this because already in the name?
sparsity?

In [20]:
# now we'll load the model
model_name = "google/gemma-2-2b"

# Note: The warnings below also seem to be present on a test script in SAE Lens for gemma-2-2b: https://github.com/jbloomAus/SAELens/blob/363f9a66e0cbf88ed6afc4b5a24ace77464839f9/scripts/joseph_curt_pairing_gemma_scope_saes.ipynb#L123
#
# WARNING:root:You tried to specify center_unembed=True for a model using logit softcap,
#              but this can't be done! Softcapping is not invariant upon adding a
#              constantSetting center_unembed=False instead.
#
# WARNING:root:You are not using LayerNorm, so the writing weights can't be centered!
#              Skipping
#
model = sae_lens.HookedSAETransformer.from_pretrained(
    model_name,
    device=device,
)

config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Loaded pretrained model google/gemma-2-2b into HookedTransformer


In [21]:
# can inspect the model config, which is often useful
model.cfg

HookedTransformerConfig:
{'act_fn': 'gelu_pytorch_tanh',
 'attention_dir': 'causal',
 'attn_only': False,
 'attn_scale': 16.0,
 'attn_scores_soft_cap': 50.0,
 'attn_types': ['global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',


In [22]:
# Note: First replicating exactly what's in a tutorial or paper is important
#       to make sure can go back one at a time if off and that understand
#       how each step is sanity checked

# Generate text using the model
# note: return type matches input
generated_text: str = model.generate(
    "Would you be able to travel through time using a wormhole?",
    max_new_tokens=10,  # Limit the number of new tokens to generate
    # temperature=0.7,    # Add some randomness to generation
    # do_sample=True      # Use sampling instead of greedy decoding
    verbose=True,  # Show progress during generation
)

# Print the generated text
print(f"{generated_text=}")

  0%|          | 0/10 [00:00<?, ?it/s]

generated_text='Would you be able to travel through time using a wormhole? There have been different models of vast wormhole in'


In [23]:
# Analyze the generated text
analyze_generated_text = False
if analyze_generated_text:

    logits, cache = model.run_with_cache(generated_text)
    log_probs = logits.log_softmax(dim=-1)

    # Visualize token probabilities
    cv.logits.token_log_probs(
        token_indices=generated_text[0],
        log_probs=log_probs,
        to_string=model.to_string,
    )

In [24]:
# and let's do an example of seeing answer to test
transformer_lens.utils.test_prompt(
    prompt="Jill threw the ball to Jack. Jack threw the ball to Will. Will threw the ball back to",
    answer=" Jill",
    model=model,
    prepend_space_to_answer=True,  # default
    print_details=True,  # default
    prepend_bos=None,  # default
    top_k=10,  # default
)

Tokenized prompt: ['<bos>', 'Jill', ' threw', ' the', ' ball', ' to', ' Jack', '.', ' Jack', ' threw', ' the', ' ball', ' to', ' Will', '.', ' Will', ' threw', ' the', ' ball', ' back', ' to']
Tokenized answer: [' Jill']


Performance on answer token:
Rank: 0        Logit: 27.67 Prob: 68.66% Token: | Jill|

Top 0th token. Logit: 27.67 Prob: 68.66% Token: | Jill|
Top 1th token. Logit: 26.76 Prob: 27.73% Token: | Jack|
Top 2th token. Logit: 22.38 Prob:  0.35% Token: | the|
Top 3th token. Logit: 21.76 Prob:  0.19% Token: | Jake|
Top 4th token. Logit: 21.59 Prob:  0.16% Token: | J|
Top 5th token. Logit: 21.53 Prob:  0.15% Token: | Joe|
Top 6th token. Logit: 21.17 Prob:  0.10% Token: | him|
Top 7th token. Logit: 21.17 Prob:  0.10% Token: | Jane|
Top 8th token. Logit: 21.05 Prob:  0.09% Token: |...|
Top 9th token. Logit: 21.04 Prob:  0.09% Token: | Alice|


Ranks of the answer tokens: [(' Jill', 0)]

#### Using the hooked SAE transformer

In [25]:
# note: we'll assume these SAEs have small error term since they're from GemmaScope

# SAEs don't reconstruct activation perfectly, so if you attach an SAE and want the model to stay performant, you need to use the error term.
# This is because the SAE will be used to modify the forward pass, and if it doesn't reconstruct the activations well, the outputs may be effected.
# Good SAEs have small error terms but it's something to be mindful of.

sae.use_error_term  # If use error term is set to false, we will modify the forward pass by using the sae.

False

In [50]:
# hooked SAE Transformer will enable us to get the feature activations from the SAE
# example_prompt = "Jill threw the ball to Jack. Jack threw the ball to Will. Will threw the ball back to"

# note: it's trained on the pile right? no that's just the SAE not gemma 2 itself
example_prompt = (
    'The scheming, lying, and misaligned model said'
)
logits, cache = model.run_with_cache_with_saes(
    example_prompt,
    saes=[sae],
)

In [51]:
# see what's in the cache related to SAE
# print(tabulate.tabulate([(k, v.shape) for k, v in cache.items() if "sae" in k]))

# ex: because this SAE is operating on the residual stream
assert sae.cfg.d_in == model.cfg.d_model

example_prompt_tokens = model.to_tokens(example_prompt)

print(f"Relevant numbers:")
print(f"- {example_prompt_tokens.shape=}")
print(f"- {model.cfg.d_model=}")
print(f"- {sae.cfg.d_sae=}")

Relevant numbers:
- example_prompt_tokens.shape=torch.Size([1, 12])
- model.cfg.d_model=2304
- sae.cfg.d_sae=16384


In [52]:
# show everything not related to SAE (note it's essentially just every operation hooked)
# print(tabulate.tabulate([(k, v.shape) for k, v in cache.items() if "sae" not in k]))

#### What feature explanations do we have for this SAE?

* Explanations are generated by GPT-4o-mini looking at activating examples in `ThePile`

In [30]:
import requests
from typing import Any

import IPython.display


# ex: https://www.neuronpedia.org/gemma-2-2b/25-gemmascope-res-16k/3742
#     from url directly


# note: not all SAEs in neuronpedia yet, so we get the closest one
class NeuronpediaConstants:

    MODEL_ID = "gemma-2-2b"

    # note: this must be same width as the `sae_id` we're using for the loaded SAE, otherwise there won't be autointerp explanations available
    # SAE_ID = "25-gemmascope-res-16k"

    # copied exactly from gemmascope colab tutorial https://colab.research.google.com/drive/17dQFYUYnuKnP6OwQPH9v_GSYUW5aj-Rp#scrollTo=2-i7YRVLgKoT
    SAE_ID = "20-gemmascope-res-16k"

    EXPORT_URL = "https://www.neuronpedia.org/api/explanation/export"


def get_neuronpedia_dashboard_html_url(
    feature_index: int,
    model_id: str = NeuronpediaConstants.MODEL_ID,
    sae_id: str = NeuronpediaConstants.SAE_ID,
) -> str:
    """Create URL for getting an individual feature's HTML, rendered via IFrame"""
    return (
        f"https://www.neuronpedia.org/{model_id}/{sae_id}/{feature_index}"
        "?embed=true&embedexplanation=true&embedplots=true&embedtest=true&height=300"
    )


def show_neuronpedia_dashboard(
    feature_index: int,
    model_id: str = NeuronpediaConstants.MODEL_ID,
    sae_id: str = NeuronpediaConstants.SAE_ID,
) -> None:
    """Show the neuronpedia dashboard for a given feature index"""

    html_url = get_neuronpedia_dashboard_html_url(feature_index=feature_index)

    display(IPython.display.IFrame(html_url, width=800, height=500))


# API: https://www.neuronpedia.org/api-doc
#
# note: so neuronpedia is also a store of autointerp explanations
def get_neuronpedia_explanations(
    model_id: str = NeuronpediaConstants.MODEL_ID,
    sae_id: str = NeuronpediaConstants.SAE_ID,
) -> pd.DataFrame:
    """Get explanations from neuronpedia for a given model and sae."""

    url = "https://www.neuronpedia.org/api/explanation/export"

    payload = {
        "modelId": model_id,
        "saeId": sae_id,
    }
    headers = {"Content-Type": "application/json"}

    response = requests.get(url, params=payload, headers=headers)

    response.raise_for_status()

    # - explanations
    # - explanationsCount
    response_json: list[dict[str, Any]] = response.json()

    print(f"{len(response_json)=} {type(response_json)=}")

    num_explanations = len(response_json)
    print(f"{num_explanations=}")

    print("Example explanation:")
    python_utils.print_json(response_json[0])

    # convert to pandas
    explanations_df = pd.DataFrame(response_json)

    # rename index to "feature"
    explanations_df = explanations_df.rename(columns={"index": "feature"})

    # explanations_df["feature"] = explanations_df["feature"].astype(int)
    explanations_df["description"] = explanations_df["description"].apply(
        lambda x: x.lower()
    )

    return explanations_df


# get all explanations
explanations_df = get_neuronpedia_explanations()

# TODO(bschoen): How are features not unique?
print(explanations_df.iloc[0])

# okay so only gpt-4o-mini for now
explanations_df["explanationModelName"].value_counts()

len(response_json)=17008 type(response_json)=<class 'list'>
num_explanations=17008
Example explanation:
{
  "modelId": "gemma-2-2b",
  "layer": "20-gemmascope-res-16k",
  "index": "14403",
  "description": "phrases or sentences that introduce lists, examples, or elaborations, often followed by commas.",
  "explanationModelName": "claude-3-5-sonnet-20240620",
  "typeName": "oai_token-act-pair"
}
modelId                                                        gemma-2-2b
layer                                               20-gemmascope-res-16k
feature                                                             14403
description             phrases or sentences that introduce lists, exa...
explanationModelName                           claude-3-5-sonnet-20240620
typeName                                               oai_token-act-pair
Name: 0, dtype: object


,count
explanationModelName,
gpt-4o-mini,16384
claude-3-5-sonnet-20240620,317
gpt-3.5-turbo,303
gemini-1.5-flash,2
gemini-1.5-pro,1
gpt-4o,1


##### Searching for specific features

In [31]:
target_description = "decept"

df_target_descriptions = explanations_df.loc[
    explanations_df.description.str.contains(target_description)
]

df_target_descriptions[["feature", "description"]]

,feature,description
5088,6382,situations involving deception or trickery
5136,6458,words or phrases related to deception or manip...
6741,8751,phrases related to deception and misleading in...
6866,8927,terms related to artificiality and deception
15298,2746,terms and concepts related to fraudulent activ...


In [ ]:
feature_index = explanations_df["feature"].iloc[5088]

html_url = get_neuronpedia_dashboard_html_url(feature_index=feature_index)

IPython.display.IFrame(html_url, width=800, height=500)

In [53]:
# let's look at which features fired
cache_id = "blocks.20.hook_resid_post.hook_sae_acts_post"
# cache_id = "blocks.25.hook_resid_post.hook_sae_acts_post"

In [54]:
# # torch.Size([1, <prompt_length>, <sae_size>]) -> (<prompt_length>, <sae_size>)
sae_activations = cache[cache_id][0]

print(f"{sae_activations.shape=}")

sae_activations.shape=torch.Size([12, 16384])


In [55]:
# get the max activating feature at each position
#
# both of size (<prompt-length>, )
activation_values, feature_indices = sae_activations.max(-1)

print(f"{activation_values.shape=}")

activation_values.shape=torch.Size([12])


In [56]:
cache[cache_id].shape

torch.Size([1, 12, 16384])

In [57]:
show_top_k = False

if show_top_k:

    # let's print the top 5 features and how much they fired
    topk_count = 5

    # both are (<prompt-length>, <topk_count>)
    activation_values, feature_indices = torch.topk(sae_activations, topk_count)

    print(f"{activation_values.shape=}")
    print(activation_values)

    print(f"{feature_indices.shape=}")
    print(feature_indices)

In [58]:
sae_activations.shape

torch.Size([12, 16384])

In [59]:
# convert prompt to tokens so we can show it alongside features / feature index
example_prompt_as_tokens = model.to_tokens(example_prompt)

print(f"{example_prompt_as_tokens.shape=}")

example_prompt_as_tokens

example_prompt_as_tokens.shape=torch.Size([1, 12])


tensor([[     2,    651,   4288,   3357, 235269,  17347, 235269,    578,   2586,
           1933,   2091,   1180]], device='cuda:0')

In [60]:
# note: just maps to first description, there may be multiple
explanations_df["feature"] = explanations_df["feature"].astype(int)

feature_index_to_description = explanations_df.set_index("feature")[
    "description"
].to_dict()

print(f"{len(feature_index_to_description)=}")

len(feature_index_to_description)=16383


In [61]:
# should be time travel related (for layer 20)
# feature_index_to_description[10004]

In [62]:
# TODO(bschoen): Need to come back to this, for now skipping autointerp

import math

# let's separately also show the topk
# let's print the top 5 features and how much they fired
topk_count = 5

# both are (<prompt-length>, <topk_count>)
activation_values_topk, feature_indices_topk = torch.topk(sae_activations, topk_count)

# shape: (batch, <prompt-length>)
example_prompt_as_tokens = model.to_tokens(example_prompt)

# convert to a dataframe
rows = []

# note: `i` is position in prompt (tokenized)
for i in range(example_prompt_as_tokens.shape[-1]):

    token_int = example_prompt_as_tokens[:, i].item()
    token_str = model.to_single_str_token(token_int)

    activation_value = activation_values[i].item()
    feature_index = feature_indices[i].item()

    num_explanations = (explanations_df["feature"].astype(int) == feature_index).sum()
    description = feature_index_to_description.get(feature_index, "NONE")

    rows.append(
        {
            "position": i,
            "token_int": token_int,
            "token_str": token_str,
            "activation_value": activation_value,
            "feature_index": feature_index,
            "num_explanations": num_explanations,
            "description": description,
        }
    )

    print(f"[{i}] {token_str}")

    for topk_index in range(topk_count):

        activation_value_topk = activation_values_topk[i, topk_index].item()
        feature_index_topk = feature_indices_topk[i, topk_index].item()

        # lookup description
        description = feature_index_to_description.get(feature_index, "NONE")

        print(f" - {activation_value_topk:.2f} [{feature_index_topk}] := {description}")

df = pd.DataFrame(rows)

[0] <bos>
 - 2028.81 [6631] := the beginning of a text or important markers in a document
 - 781.40 [743] := the beginning of a text or important markers in a document
 - 534.86 [5052] := the beginning of a text or important markers in a document
 - 264.19 [16057] := the beginning of a text or important markers in a document
 - 252.53 [9479] := the beginning of a text or important markers in a document
[1] The
 - 155.71 [11795] := the phrase "the" at the start of sentences
 - 95.88 [11527] := the phrase "the" at the start of sentences
 - 89.36 [3458] := the phrase "the" at the start of sentences
 - 41.42 [11687] := the phrase "the" at the start of sentences
 - 39.72 [9768] := the phrase "the" at the start of sentences
[2]  sche
 - 104.56 [6596] := mentions of the pronoun "she."
 - 89.59 [9091] := mentions of the pronoun "she."
 - 55.34 [4388] := mentions of the pronoun "she."
 - 48.85 [8366] := mentions of the pronoun "she."
 - 38.20 [9768] := mentions of the pronoun "she."
[3] ming
 -

In [ ]:
df[["token_str", "activation_value", "feature_index", "description"]]

In [ ]:
get_neuronpedia_dashboard_html_url(feature_index=14325)

In [ ]:
print(example_prompt)

In [ ]:
for activation_value, feature_index, token in zip(
    activation_values,
    feature_indices,
    example_prompt_as_tokens,
):
    print(f"{activation_value.item()=:.4f}")
    print(f"{feature_index.item()=}")

    feature_index_int = feature_index.item()

    # note: there could just legitimately be features without explanations
    #       that just gives the "this feature has no known explanations"
    had_feature_index_in_explanations_df = (
        explanations_df["feature"].astype(int) == feature_index_int
    ).sum() > 0

    print(f"{had_feature_index_in_explanations_df=}")

    # if had_feature_index_in_explanations_df:

    show_neuronpedia_dashboard(feature_index=feature_index_int)

## Setup Steering

In [41]:
# we'll look at the one for lying

feature_index = 817

description = feature_index_to_description[feature_index]

print(description)

instances of deceit or false testimony


In [92]:
steering_vector = sae.W_dec[feature_index]

# note: "you may want to experiment with the sampling coefficient"
coeff = 30000
sampling_kwargs = {}# dict(temperature=1.0, top_p=0.1, freq_penalty=1.0)

### Setup Hooks

In [43]:
# only used to get shape
sae_out = sae.decode(sae_activations)

print(f"{sae_out.shape=}")

sae_out.shape=torch.Size([54, 2304])


In [93]:
steering_on = False


def steering_hook(resid_pre, hook):
    if resid_pre.shape[1] == 1:
        return

    position = sae_out.shape[1]
    if steering_on:
        # using our steering vector and applying the coefficient
        resid_pre[:, : position - 1, :] += coeff * steering_vector


def hooked_generate(prompt_batch, fwd_hooks=[], seed=None, **kwargs):
    if seed is not None:
        torch.manual_seed(seed)

    with model.hooks(fwd_hooks=fwd_hooks):
        tokenized = model.to_tokens(prompt_batch)
        result = model.generate(
            stop_at_eos=False,  # avoids a bug on MPS
            input=tokenized,
            max_new_tokens=10,
            do_sample=True,
            **kwargs,
        )
    return result

In [94]:
def run_generate(example_prompt):
    model.reset_hooks()
    editing_hooks = [(f"blocks.20.hook_resid_post", steering_hook)]
    res = hooked_generate(
        [example_prompt] * 10, editing_hooks, seed=None, **sampling_kwargs
    )

    # Print results, removing the ugly beginning of sequence token
    res_str = model.to_string(res[:, 1:])
    print(("\n\n" + "-" * 80 + "\n\n").join(res_str))

In [96]:
steering_on = False
run_generate("Options: (A): Lie (B): Tell the truth\nAnswer:")

  0%|          | 0/10 [00:00<?, ?it/s]

Options: (A): Lie (B): Tell the truth
Answer: (C)
The word “options” is

--------------------------------------------------------------------------------

Options: (A): Lie (B): Tell the truth
Answer: (C) Lie

<h3>In which period of

--------------------------------------------------------------------------------

Options: (A): Lie (B): Tell the truth
Answer: Lie
Explanation:

When he is lying outside

--------------------------------------------------------------------------------

Options: (A): Lie (B): Tell the truth
Answer: See A
Who actually said what?
John

--------------------------------------------------------------------------------

Options: (A): Lie (B): Tell the truth
Answer: (B)

I would assume that you indicated

--------------------------------------------------------------------------------

Options: (A): Lie (B): Tell the truth
Answer: (D) You have free will option.


--------------------------------------------------------------------------------

Options: (A): Lie (